# Foreword

In this notebook we show how to use the Talos tool for automatic network optimization (and, in general, hyperparameter tuning). This example uses the breast cancer dataset which we have already seen in the course and is completely self contained. However if you want to further understand what's going on please refer to:

* [Talos library official website](https://pypi.org/project/talos/)
* [Talos github repository, with examples](https://github.com/autonomio/talos)
* [Talos documentation](https://autonomio.github.io/talos/#/README?id=quick-start)

# Setup(s)

## Standard libraries setup

In [ ]:
#very common libraries, that we for sure are using
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Talos setup

In [ ]:
#making sure talos is installed
!pip install talos

#importing the library
import talos

## Seed setup

In [ ]:
from numpy.random import seed
myseed = 0
seed(myseed)
tf.random.set_seed(myseed)

## Data setup

In [ ]:
#libraries for this block
import sklearn.datasets
from sklearn.model_selection import StratifiedShuffleSplit

# loading data
from sklearn.datasets import load_breast_cancer
bcancer = load_breast_cancer()
y = bcancer.target
X = pd.DataFrame(bcancer.data, columns=bcancer.feature_names)

# normalizing
X = (X - X.mean())/X.std()

# Talos workflow

## Hyperparamaters to be explored

This part becomes central. We define the space (i.e. the amount of combinations) that we are going to explore.

In [ ]:
#talos requires a dictionary
my_parameters = {
    #these hyperparameters need to be optimized
    'first_layer': [12, 24, 48],
    'second_layer': [12, 24, 48],
    'LR' : [0.0001, 0.1, 10]
}

## Train function (default, without Talos)

We report below a basic function used to train and return a model. It doesn not accept any hyperparameter (all values are hard coded). It will not be used in the example, but it serves as reference for when we integrate Talos.

In [ ]:
#libraries for this block
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# a function to declare and train the network. Returns the trained model
def train_net_default(x_train, y_train, x_val, y_val):

  #this depends on the input data
  input_shape = (x_train.shape[1],)

  #a simple neural network with two hidden layers
  model = Sequential()
  model.add(Dense(10, activation='relu'))
  model.add(Dense(5, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  #instantiating the optimizer, compiling, training
  opt = SGD(learning_rate=0.1)
  model.compile(optimizer=opt, loss='binary_crossentropy')
  history = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                      epochs=100, verbose=0)

  return(model)


## Train function, with Talos

In [ ]:
# a function to declare and train the network, accepting a dictionary of
# hyperparameters. It must return both the trained model and the training history
def train_net_talos(x_train, y_train, x_val, y_val, par):

  #this depends on the input data
  input_shape = (X.shape[1],)

  #a simple neural network with two hidden layers
  model = Sequential()
  model.add(Dense(par['first_layer'], activation='relu'))
  model.add(Dense(par['second_layer'], activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  #instantiating the optimizer, compiling, training
  opt = SGD(learning_rate=par['LR'])
  model.compile(optimizer=opt, loss='binary_crossentropy')
  history = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val],
                      epochs=100, verbose=0)

  #returning both history and model, in that order
  return(history, model)

## Running a Talos "scan"

In this simple example Talos will check all the available combinations, once.

In [ ]:
#note: Talos does not directly support Pandas dataframes. However, it's quite
#straightforward to obtain a table using .values
t = talos.Scan(x=X.values, y=y, params=my_parameters, model=train_net_talos, experiment_name='breast_cancer')

We just obtained a Scan object, which contain in the `.data` field information on all the tested combinations.

In [ ]:
print(type(t))
print(t.data.shape)

Taking a look at the results

In [ ]:
t.data

Extracting the best configuration

In [ ]:
#a local copy, for easier notation
df = t.data

#printing the row with lowest validation loss
df[df.val_loss == df.val_loss.min()]

Extracting the best performing model

In [ ]:
#I need to specify what is the criterion (i.e. the metric) used to define the "best" model.
#Moreover, "asc" has to be True for the case where the metric is something to be minimized.
best_model = t.best_model(metric='val_loss', asc=True)
print(type(best_model))

# Further steps

The code above is a **very** minimal example and works as a starting point. Stuff to consider:

* each combination of hyperparameter is trained once, with a 70/30 default split. Using `.evaluate_models()` it's possible to do a proper k-fold crossvalidation (see [scan documentation](https://autonomio.github.io/talos/#/Scan), search "evaluate_models")
* the default approach of trying all the combinations can become unfeasible very quickly. The `Scan` function supports several policies for sampling a subset of the hyperparameter space. See the [Towardsdatascience's tutorial](https://towardsdatascience.com/tune-the-hyperparameters-of-your-deep-learning-networks-in-python-using-keras-and-talos-2a2a38c5ac31) for a more in-depth example